In [4]:
!pip install plotly --quiet

In [5]:
# EXECUTA AIXÒ PRIMER a una cel·la separada
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.offline import init_notebook_mode, iplot
import plotly.io as pio

# Configuració específica per Google Colab
pio.renderers.default = "colab"
init_notebook_mode(connected=True)

# Test ràpid per veure si funciona
fig_test = go.Figure(data=go.Scatter(x=[1, 2, 3], y=[4, 5, 6]))
fig_test.update_layout(title="Test - Si veus això, Plotly funciona!")
fig_test.show()

In [6]:
# Mount drive
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Llegim el dataset
df = pd.read_csv("/content/drive/MyDrive/UOC/3r/Visualització/PRA1/amazon.csv")

In [8]:
# Netejem les columnes de preu
def clean_price(price_str):
    if isinstance(price_str, str):
        # Eliminar el símbol de rupia i qualsevol coma
        return float(re.sub(r'[₹,]', '', price_str))
    return price_str

def to_euro(rupee):
    if isinstance(rupee, str):
        return (clean_price(rupee)) / 95
    else:
        return rupee

df['actual_price'] = df['actual_price'].apply(to_euro)
df['discounted_price']  = df['discounted_price'].apply(to_euro)
df['discount_percentage'] = df['discount_percentage'].str.rstrip('%').astype(float)
# Separem les categories en columnes
category_split = df['category'].str.split('|', expand=True)
category_split.columns = [f'category_level_{i+1}' for i in range(category_split.shape[1])]

# Afegim aquestes columnes al dataframe original
df = pd.concat([df, category_split], axis=1)

In [9]:
# Example: count records per category
counts = df['category_level_1'].value_counts()

# Filter categories with at least 20 records
categories_to_keep = counts[counts >= 20].index.tolist()

# Filter df to keep only categories with enough records
df = df[df['category_level_1'].isin(categories_to_keep)]

In [10]:
# Visualització Interactiva de Dades E-commerce
# Autor: [El teu nom]
# Data: Maig 2025

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

# Configuració inicial per Colab
import plotly.io as pio
pio.renderers.default = "colab"  # Específic per Google Colab
pyo.init_notebook_mode(connected=True)

# ============================================================================
# 1. INTRODUCCIÓ I OBJECTIUS
# ============================================================================
print("="*80)
print("ANÀLISI VISUAL DE DADES E-COMMERCE")
print("="*80)
print("""
PREGUNTES DE RECERCA:
1. Quina és la relació entre preu, descompte i valoració dels productes?
2. Com es distribueixen les categories de productes per popularitat i preu?
3. Hi ha patrons estacionals o tendències en les valoracions?
4. Quins factors influeixen més en la satisfacció del client?

OBJECTIUS:
- Identificar oportunitats de negoci basades en preus i valoracions
- Descobrir categories amb millor rendiment
- Analitzar l'efectivitat dels descomptes
- Proporcionar insights accionables per a la gestió del catàleg
""")

# ============================================================================
# 2. CÀRREGA I PREPARACIÓ DE DADES
# ============================================================================
def load_and_prepare_data(df):
    """
    Prepara les dades per a la visualització
    """
    # Neteja bàsica
    df = df.copy()

    # Convertir preus a numèric
    df['discounted_price'] = pd.to_numeric(df['discounted_price'], errors='coerce')
    df['actual_price'] = pd.to_numeric(df['actual_price'], errors='coerce')
    df['discount_percentage'] = pd.to_numeric(df['discount_percentage'], errors='coerce')
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    df['rating_count'] = pd.to_numeric(df['rating_count'], errors='coerce')

    # Omplir valors nuls amb valors per defecte
    df['rating_count'] = df['rating_count'].fillna(1)  # Si no hi ha rating_count, assumim 1
    df['discount_percentage'] = df['discount_percentage'].fillna(0)  # Si no hi ha descompte, assumim 0%

    # Eliminar files amb valors nuls crítics
    df = df.dropna(subset=['discounted_price', 'rating'])

    # Assegurar que rating_count sigui positiu per a la visualització
    df['rating_count'] = df['rating_count'].clip(lower=1)

    # Crear variables derivades
    df['price_range'] = pd.cut(df['discounted_price'],
                              bins=[0, 20, 50, 100, 200, np.inf],
                              labels=['Molt Baix (<20)', 'Baix (20-50)', 'Mitjà (50-100)',
                                     'Alt (100-200)', 'Molt Alt (>200)'])

    df['rating_category'] = pd.cut(df['rating'],
                                  bins=[0, 3, 4, 4.5, 5],
                                  labels=['Baix (<3)', 'Mitjà (3-4)', 'Bo (4-4.5)', 'Excel·lent (>4.5)'])

    df['discount_range'] = pd.cut(df['discount_percentage'],
                                 bins=[0, 10, 25, 50, 100],
                                 labels=['Baix (<10%)', 'Mitjà (10-25%)', 'Alt (25-50%)', 'Molt Alt (>50%)'])

    # Categoria principal (assumint que category_level_1 és la més general)
    df['main_category'] = df['category_level_1'].fillna(df['category'])

    return df

# ============================================================================
# 3. FUNCIONS DE VISUALITZACIÓ
# ============================================================================

def create_price_rating_scatter(df):
    """
    Scatter plot interactiu: Preu vs Valoració amb informació de categoria
    """
    # Mostra de dades per rendiment (ajusta segons necessitat)
    df_sample = df.sample(min(5000, len(df))) if len(df) > 5000 else df

    # Assegurar que no hi ha valors NaN en les columnes crítiques
    df_sample = df_sample.dropna(subset=['discounted_price', 'rating', 'rating_count'])

    # Normalitzar la mida dels punts (entre 3 i 20)
    df_sample['size_normalized'] = ((df_sample['rating_count'] - df_sample['rating_count'].min()) /
                                   (df_sample['rating_count'].max() - df_sample['rating_count'].min()) * 17 + 3)

    fig = px.scatter(df_sample,
                    x='discounted_price',
                    y='rating',
                    color='main_category',
                    size='size_normalized',
                    hover_data=['product_name', 'discount_percentage', 'actual_price', 'rating_count'],
                    title="Relació entre Preu i Valoració per Categoria",
                    labels={
                        'discounted_price': 'Preu amb Descompte (€)',
                        'rating': 'Valoració (1-5)',
                        'main_category': 'Categoria Principal',
                        'size_normalized': 'Popularitat'
                    })

    fig.update_layout(
        template='plotly_white',
        height=600,
        showlegend=True,
        title_x=0.5
    )

    return fig

def create_category_analysis(df):
    """
    Anàlisi per categories amb múltiples mètriques
    """
    # Agregació per categoria
    cat_stats = df.groupby('main_category').agg({
        'rating': 'mean',
        'rating_count': 'sum',
        'discounted_price': 'mean',
        'discount_percentage': 'mean',
        'product_id': 'count'
    }).round(2)

    cat_stats.columns = ['Valoració Mitjana', 'Total Valoracions', 'Preu Mitjà', 'Descompte Mitjà %', 'Num Productes']
    cat_stats = cat_stats.sort_values('Total Valoracions', ascending=False).head(15)

    # Subplot amb múltiples gràfics
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Valoració Mitjana per Categoria', 'Preu Mitjà per Categoria',
                       'Descompte Mitjà per Categoria', 'Número de Productes per Categoria'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    categories = cat_stats.index

    # Gràfic 1: Valoració mitjana
    fig.add_trace(
        go.Bar(x=categories, y=cat_stats['Valoració Mitjana'],
               name='Valoració', marker_color='lightblue'),
        row=1, col=1
    )

    # Gràfic 2: Preu mitjà
    fig.add_trace(
        go.Bar(x=categories, y=cat_stats['Preu Mitjà'],
               name='Preu', marker_color='lightgreen'),
        row=1, col=2
    )

    # Gràfic 3: Descompte mitjà
    fig.add_trace(
        go.Bar(x=categories, y=cat_stats['Descompte Mitjà %'],
               name='Descompte', marker_color='lightcoral'),
        row=2, col=1
    )

    # Gràfic 4: Número de productes
    fig.add_trace(
        go.Bar(x=categories, y=cat_stats['Num Productes'],
               name='Productes', marker_color='lightyellow'),
        row=2, col=2
    )

    fig.update_layout(
        height=800,
        showlegend=False,
        title_text="Anàlisi Comparativa per Categories",
        title_x=0.5,
        template='plotly_white'
    )

    # Rotar etiquetes de l'eix X
    fig.update_xaxes(tickangle=45)

    return fig

def create_discount_effectiveness(df):
    """
    Anàlisi de l'efectivitat dels descomptes
    """
    fig = px.box(df, x='discount_range', y='rating',
                title="Efectivitat dels Descomptes en les Valoracions",
                labels={
                    'discount_range': 'Rang de Descompte',
                    'rating': 'Valoració (1-5)'
                })

    fig.update_layout(
        template='plotly_white',
        height=500,
        title_x=0.5
    )

    return fig

def create_price_distribution(df):
    """
    Distribució de preus amb histogram interactiu
    """
    fig = px.histogram(df, x='discounted_price', nbins=50,
                      title="Distribució de Preus dels Productes",
                      labels={'discounted_price': 'Preu amb Descompte (€)', 'count': 'Número de Productes'})

    fig.add_vline(x=df['discounted_price'].median(), line_dash="dash", line_color="red",
                 annotation_text=f"Mediana: {df['discounted_price'].median():.2f}€")

    fig.update_layout(
        template='plotly_white',
        height=400,
        title_x=0.5
    )

    return fig

def create_rating_trends(df):
    """
    Tendències de valoracions per rang de preus
    """
    custom_order = labels=['Molt Baix (<20)', 'Baix (20-50)', 'Mitjà (50-100)',
                                     'Alt (100-200)', 'Molt Alt (>200)']  # Adjust to your actual categories

    fig = px.violin(
        df,
        x='price_range',
        y='rating',
        title="Distribució de Valoracions per Rang de Preus",
        labels={
            'price_range': 'Rang de Preus',
            'rating': 'Valoració (1-5)'
        },
        category_orders={'price_range': custom_order}
    )

    fig.update_layout(
        template='plotly_white',
        height=500,
        title_x=0.5
    )

    return fig

# ============================================================================
# 4. DASHBOARD PRINCIPAL
# ============================================================================

def create_dashboard(df):
    """
    Crea el dashboard complet amb totes les visualitzacions
    """
    print("Preparant les dades...")
    df_clean = load_and_prepare_data(df)

    print(f"Dades processades: {len(df_clean)} productes")
    print(f"Categories principals: {df_clean['main_category'].nunique()}")
    print("-" * 50)

    # Estadístiques bàsiques
    print("ESTADÍSTIQUES BÀSIQUES:")
    print(f"Preu mitjà: {df_clean['discounted_price'].mean():.2f}€")
    print(f"Valoració mitjana: {df_clean['rating'].mean():.2f}/5")
    print(f"Descompte mitjà: {df_clean['discount_percentage'].mean():.1f}%")
    print("-" * 50)

    # Crear visualitzacions
    print("Creant visualitzacions...")

    fig1 = create_price_rating_scatter(df_clean)
    fig2 = create_category_analysis(df_clean)
    fig3 = create_discount_effectiveness(df_clean)
    fig4 = create_price_distribution(df_clean)
    fig5 = create_rating_trends(df_clean)

    # Mostrar gràfics amb configuració específica per Colab
    print("Mostrant dashboard...")

    print("1. Gràfic Preu vs Valoració:")
    fig1.show(renderer="colab")

    print("2. Anàlisi per Categories:")
    fig2.show(renderer="colab")

    print("3. Efectivitat dels Descomptes:")
    fig3.show(renderer="colab")

    print("4. Distribució de Preus:")
    fig4.show(renderer="colab")

    print("5. Tendències de Valoracions:")
    fig5.show(renderer="colab")

    return df_clean

# ============================================================================
# 5. INSIGHTS I CONCLUSIONS
# ============================================================================

def generate_insights(df):
    """
    Genera insights automàtics basats en les dades
    """
    print("="*80)
    print("INSIGHTS I CONCLUSIONS")
    print("="*80)

    # Top categories per valoració
    top_rated = df.groupby('main_category')['rating'].mean().sort_values(ascending=False).head(5)
    print("TOP 5 CATEGORIES MILLOR VALORADES:")
    for cat, rating in top_rated.items():
        print(f"  • {cat}: {rating:.2f}/5")

    print()

    # Correlació preu-valoració
    corr = df['discounted_price'].corr(df['rating'])
    print(f"CORRELACIÓ PREU-VALORACIÓ: {corr:.3f}")
    if abs(corr) < 0.1:
        print("  → No hi ha correlació significativa entre preu i valoració")
    elif corr > 0:
        print("  → Els productes més cars tendeixen a tenir millors valoracions")
    else:
        print("  → Els productes més barats tendeixen a tenir millors valoracions")

    print()

    # Efectivitat dels descomptes
    discount_effect = df.groupby('discount_range')['rating'].mean()
    print("EFECTIVITAT DELS DESCOMPTES:")
    for range_name, rating in discount_effect.items():
        print(f"  • {range_name}: {rating:.2f}/5")

    print()
    print("RECOMANACIONS:")
    print("• Analitza les categories amb millor valoració per identificar oportunitats")
    print("• Revisa l'estratègia de preus basant-te en la correlació trobada")
    print("• Optimitza els descomptes segons la seva efectivitat en les valoracions")
    print("• Considera la popularitat (rating_count) juntament amb les valoracions")

# ============================================================================
# 6. ÚS DEL CODI
# ============================================================================

"""
INSTRUCCIONS D'ÚS:

1. Carrega el teu dataset:
   df = pd.read_csv('el_teu_fitxer.csv')

2. Executa el dashboard:
   df_processed = create_dashboard(df)

3. Genera insights:
   generate_insights(df_processed)

PERSONALITZACIÓ:
- Ajusta els rangs de preus a les funcions segons les teves dades
- Modifica els colors i estils dels gràfics
- Afegeix més mètriques o visualitzacions segons necessitat
- Canvia la mostra de dades si el rendiment és lent

ELEMENTS INTERACTIUS INCLOSOS:
- Zoom i pan en tots els gràfics
- Hover amb informació detallada
- Filtratge per categories amb la llegenda
- Selecció i brushing en scatter plots
"""

ANÀLISI VISUAL DE DADES E-COMMERCE

PREGUNTES DE RECERCA:
1. Quina és la relació entre preu, descompte i valoració dels productes?
2. Com es distribueixen les categories de productes per popularitat i preu?
3. Hi ha patrons estacionals o tendències en les valoracions?
4. Quins factors influeixen més en la satisfacció del client?

OBJECTIUS:
- Identificar oportunitats de negoci basades en preus i valoracions
- Descobrir categories amb millor rendiment
- Analitzar l'efectivitat dels descomptes
- Proporcionar insights accionables per a la gestió del catàleg



"\nINSTRUCCIONS D'ÚS:\n\n1. Carrega el teu dataset:\n   df = pd.read_csv('el_teu_fitxer.csv')\n\n2. Executa el dashboard:\n   df_processed = create_dashboard(df)\n\n3. Genera insights:\n   generate_insights(df_processed)\n\nPERSONALITZACIÓ:\n- Ajusta els rangs de preus a les funcions segons les teves dades\n- Modifica els colors i estils dels gràfics\n- Afegeix més mètriques o visualitzacions segons necessitat\n- Canvia la mostra de dades si el rendiment és lent\n\nELEMENTS INTERACTIUS INCLOSOS:\n- Zoom i pan en tots els gràfics\n- Hover amb informació detallada\n- Filtratge per categories amb la llegenda\n- Selecció i brushing en scatter plots\n"

In [11]:
# 1. Carrega el teu dataset:
  #  df = pd.read_csv('el_teu_fitxer.csv')

# 2. Executa el dashboard:
df_processed = create_dashboard(df)

# 3. Genera insights:
generate_insights(df_processed)

Preparant les dades...
Dades processades: 1457 productes
Categories principals: 4
--------------------------------------------------
ESTADÍSTIQUES BÀSIQUES:
Preu mitjà: 33.02€
Valoració mitjana: 4.10/5
Descompte mitjà: 47.7%
--------------------------------------------------
Creant visualitzacions...
Mostrant dashboard...
1. Gràfic Preu vs Valoració:


2. Anàlisi per Categories:


3. Efectivitat dels Descomptes:


4. Distribució de Preus:


5. Tendències de Valoracions:


INSIGHTS I CONCLUSIONS
TOP 5 CATEGORIES MILLOR VALORADES:
  • OfficeProducts: 4.31/5
  • Computers&Accessories: 4.15/5
  • Electronics: 4.08/5
  • Home&Kitchen: 4.04/5

CORRELACIÓ PREU-VALORACIÓ: 0.121
  → Els productes més cars tendeixen a tenir millors valoracions

EFECTIVITAT DELS DESCOMPTES:
  • Baix (<10%): 4.16/5
  • Mitjà (10-25%): 4.16/5
  • Alt (25-50%): 4.11/5
  • Molt Alt (>50%): 4.06/5

RECOMANACIONS:
• Analitza les categories amb millor valoració per identificar oportunitats
• Revisa l'estratègia de preus basant-te en la correlació trobada
• Optimitza els descomptes segons la seva efectivitat en les valoracions
• Considera la popularitat (rating_count) juntament amb les valoracions
